In [2]:
''' This section builds source files needed for the 6502 disassembler to work.
    The data used as input for this task was copied from the table from
    https://www.masswerk.at/6502/6502_instruction_set.html
    by simple copy-paste magic, which was then parsed in that format.
    I did that because it was the only easily available place I could find
    a listing that included undocumented/illegal instructions in a format that
    is (relatively) easily parsed.    
    '''

datalist = list()
with open("inst6502_comp.txt", "r") as f:
    for line in f:
        data = line.strip().split('\t')
        datalist.extend([i.split(" ",1) for i in data])

datatypes = list(set([i[1] for i in datalist]))
print(datatypes)
#print(len(datatypes))
with open("../debug_itab.z80", "w") as f:
    f.write("inst6502_opcode_width_table:\n")
    for idx, i in enumerate(datatypes):
        size = 0
        if i in ("", "impl", "A"):
            size = 1
        elif i in ("zpg", "rel", "X,ind", "ind,Y", "#", "zpg,X", "zpg,Y"):
            size = 2
        elif i in ("abs", "abs,X", "abs,Y", "ind"):
            size = 3
        pass
        s = f".db {size}    ;{idx}: {i}\n"
        f.write(s)
    f.write("\n\n;Note: 1=1byteOper, 2=2byteOper, -=1byteRelOper. Other chrs are literal\n")
    f.write("\ninst6502_disassembler:\n")
    for idx, i in enumerate(datatypes):
        if i == "":
            s = ""
        elif i == "impl":
            s = ""
        elif i == "A":
            s = "A"
        elif i == "#":
            s = "#1"
        elif i == "zpg":
            s = "1"
        elif i == "rel":
            s = "-"
        elif i == "zpg,X":
            s = "1,X"
        elif i == "zpg,Y":
            s = "1,Y"
        elif i == "X,ind":
            s = "(1,X)"
        elif i == "ind,Y":
            s = "(1),Y"
        elif i == "ind":
            s = "(2)"
        elif i == "abs":
            s = "2"
        elif i == "abs,X":
            s = "2,X"
        elif i == "abs,Y":
            s = "2,Y"
        size = len(s)+1
        f.write(f'.db {size},"{s}"  ;\n')
        


    f.write("\ninst6502_disassembly_table:\n")
    for idx, i in enumerate(datalist):
        s = f'.db "{i[0].strip()[:3]}", {datatypes.index(i[1])}     ;0x{idx:02X}\n'
        f.write(s)





['', 'zpg,X', 'ind', 'abs', 'abs,X', 'X,ind', 'A', 'zpg', 'rel', '#', 'impl', 'ind,Y', 'abs,Y', 'zpg,Y']


In [1]:
import hashlib, os, importlib
import toolkit
importlib.reload(toolkit)

''' This section is intended to test some code intended to read .nes files
    and output data that ACETONES can read. This is that all-important
    ROM conversion code. Take note.'''

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


filename = "contrtst.nes" #"nestest.nes"
filebase = filename.split('.')[0]
with open(filename,"rb") as f:
    data = bytes(f.read())

print(data[:15])
isrom = False
def lo(num):
    return num & 0xFF
def hi(num):
    return (num >> 8) & 0xFF
if bytes(data[:4]) == bytes([ord('N'),ord('E'),ord('S'), 0x1A]):
    print("Is NES ROM")
    isrom = True
    prg_rom_count = int(data[4])
    chr_rom_count = int(data[5])
    print(f"PRG COUNT {prg_rom_count}, CHR COUNT {chr_rom_count}")
    hashdata = hashlib.sha256(data).digest()[:16]
    print(hashdata)
    datalen = len(data)
    convdata = tobytes(["CENROM", 0, filebase, 0, hashdata, 0, lo(datalen), hi(datalen), data])
    toolkit.export8xv(filebase, convdata)
    if os.path.exists(f"../bin/{filebase}.8xv"):
        os.remove(f"../bin/{filebase}.8xv")
    os.rename(f"{filebase}.8xv",f"../bin/{filebase}.8xv")
    a = (data[6] & 0xF0) >> 4
    b = (data[7] & 0xF0)
    print(f"Mapper ID: {a|b}")



if not isrom:
    print("Failed an NES ROM check.")



b'NES\x1a\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00'
Is NES ROM
PRG COUNT 1, CHR COUNT 1
b'\xb9h\xf1\x18\x04N\xd0\xbaj\x80\xdb\x07\xd3\xb0\xbbg'
Mapper ID: 0


In [7]:
import hashlib, os, importlib
import toolkit
''' This section creates a file meant to be used with Nestest to verify
    emulator integrity as it performs the tests used in the ROM.'''

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


timing_list = list()
status_list = list()
rega_list = list()
regy_list = list()
regx_list = list()
with open("nestest.log","r") as f:
    for line in f:
        linedata = line.split("CYC:")[1].strip()
        statusdata = line.split("P:")[1].split()[0]
        regadata = line.split("A:")[1].split()[0]
        regydata = line.split("Y:")[1].split()[0]
        regxdata = line.split("X:")[1].split()[0]
        if linedata:
            status_list.append(int(statusdata, base=16))
            timing_list.append(int(linedata))
            rega_list.append(int(regadata, base=16))
            regy_list.append(int(regydata, base=16))
            regx_list.append(int(regxdata, base=16))
print(len(timing_list))
timing_array = list()
for regx, regy, rega, sitem, item in zip(regx_list, regy_list, rega_list, status_list, timing_list):
    timing_array.append(item&0xFF)
    timing_array.append((item>>8)&0xFF)
    timing_array.append(sitem)
    timing_array.append(rega)
    timing_array.append(regx)
    timing_array.append(regy)

filebase = "timings"

toolkit.export8xv(filebase, timing_array)
if os.path.exists(f"../bin/{filebase}.8xv"):
    os.remove(f"../bin/{filebase}.8xv")
os.rename(f"{filebase}.8xv",f"../bin/{filebase}.8xv")


8991


In [5]:
import hashlib, os, importlib
import toolkit
importlib.reload(toolkit)

''' This section tests a hash-based identification system used to let
    ACETONES link related files together. The results of this test was
    later merged with the ROM conversion section somewhere above.'''

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


filename = "nestest.nes"
filebase = filename.split('.')[0]
with open("nestest.nes","rb") as f:
    data = bytes(f.read())

print(data[:15])
isrom = False
def lo(num):
    return num & 0xFF
def hi(num):
    return (num >> 8) & 0xFF
if bytes(data[:4]) == bytes([ord('N'),ord('E'),ord('S'), 0x1A]):
    print("Is NES ROM")
    isrom = True
    prg_rom_count = int(data[4])
    chr_rom_count = int(data[5])
    print(f"PRG COUNT {prg_rom_count}, CHR COUNT {chr_rom_count}")
    hashdata = hashlib.sha256(data).digest()[:16]
    print(hashdata)
    for i in range(0xC6E8,0xC700):
        print(f"{i:04X}: {data[i+16-0xC000]:02X}")

b'NES\x1a\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Is NES ROM
PRG COUNT 1, CHR COUNT 1
b'\xf6}U\xfdk<\xf0\xba\xd1\xcc\x85\xf1\xdf\rs\x9c'
C6E8: DA
C6E9: FA
C6EA: 80
C6EB: 89
C6EC: EA
C6ED: EA
C6EE: EA
C6EF: EA
C6F0: 08
C6F1: 48
C6F2: 1C
C6F3: A9
C6F4: A9
C6F5: 3C
C6F6: A9
C6F7: A9
C6F8: 5C
C6F9: A9
C6FA: A9
C6FB: 7C
C6FC: A9
C6FD: A9
C6FE: DC
C6FF: A9


In [3]:
import os, struct
'''This section converts hard-coded .pal file to ../palette.z80 source'''

#note: implementation detail sourced from graphx.h
#((uint16_t)(((((r) & 255) >> 3) << 10) | ((((g) & 255) >> 3) << 5) | (((b) & 255) >> 3)))

def convrgbto1555(rgbtuple:list|tuple):
    r,g,b = rgbtuple
    r = ((r & 0b11111000) >> 3)
    g = ((g & 0b11111000) >> 3) << 5
    b = ((b & 0b11111000) >> 3) << 10
    v = r|g|b|0x8000
    #v = ((v&255)<<8)|((v>>8)&255)   #byte swap
    return v

filedata = list()
with open("Composite_wiki.pal","rb") as f:
    filebin = f.read()
filedata = [[r,g,b] for r,g,b in struct.iter_unpack("BBB",filebin)]
print(filedata)

filedata = [convrgbto1555(i) for i in filedata]
linelen = 8
filedata = [filedata[n:n+linelen] for n in range(0,len(filedata), linelen)]

s = "paletteData:\n"

for line in filedata:
    s += ".dw "+', '.join([f'${i:04X}' for i in line])+"\n"


with open("../palette.z80", "w") as f:
    f.write(s)




[[98, 98, 98], [0, 44, 124], [17, 21, 156], [54, 3, 156], [85, 0, 124], [103, 0, 68], [103, 7, 3], [85, 28, 0], [54, 50, 0], [17, 68, 0], [0, 78, 0], [0, 76, 3], [0, 64, 68], [0, 0, 0], [0, 0, 0], [0, 0, 0], [171, 171, 171], [18, 96, 206], [61, 66, 250], [110, 41, 250], [153, 28, 206], [177, 30, 129], [177, 47, 41], [153, 74, 0], [110, 105, 0], [61, 130, 0], [18, 143, 0], [0, 141, 41], [0, 124, 129], [0, 0, 0], [0, 0, 0], [0, 0, 0], [255, 255, 255], [96, 178, 255], [141, 146, 255], [192, 120, 255], [236, 106, 255], [255, 109, 212], [255, 127, 121], [236, 155, 42], [192, 186, 0], [141, 212, 0], [96, 226, 42], [71, 224, 121], [71, 206, 212], [78, 78, 78], [0, 0, 0], [0, 0, 0], [255, 255, 255], [191, 224, 255], [209, 211, 255], [230, 201, 255], [247, 195, 255], [255, 196, 238], [255, 203, 201], [247, 215, 169], [230, 227, 151], [209, 238, 151], [191, 243, 169], [181, 242, 201], [181, 235, 238], [184, 184, 184], [0, 0, 0], [0, 0, 0], [100, 71, 70], [0, 21, 92], [18, 3, 123], [53, 0, 125], 

In [2]:
import os, hashlib, toolkit
''' This section creates a (nearly) empty ROM. Its most notable feature is that
    its CHR_ROM data section is loaded with the font data that I (shamelessly)
    "borrowed" from graphx.asm (CE C Toolchain). Link to there as follows:
    https://github.com/CE-Programming/toolchain/blob/master/src/graphx/graphx.asm
    Lightly modified to remove the weird character comments from character
    entries 1-32 and 127. Python did NOT like their presence by default and I
    didn't want to spend time trying to solve that problem the correct way.
    
    Anyhoo. I also copied the ROM conversion code from far above, made some
    tweaks, and have it generate a test ROM that does nothing but gives a way
    for me to load in CHR_ROM data. The actual program does nothing except
    exit the emulator via specially coded illegal instruction.
    '''

def tobytes(indata:list|tuple):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray)):
            for b in dat:
                outlist.append(b)
        elif isinstance(dat, (list, tuple)):
            subdata = tobytes(dat)
            for b in subdata:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)

datalist = list()
with open("charset.z80","r") as f:
    for idx,data in enumerate(f):
        newdata = data.strip()
        newdata = newdata.split()
        newdata = [i for i in newdata if i and i[0] =="$"][0]
        if not newdata:
            continue
        newdata = newdata.replace('$','').split(',')
        newdata = [int(i, 16) for i in newdata]
        entry = list()
        '''
        for d in newdata:
            databyte = [(i>>(7-n))&1 for n,i in enumerate([d]*8)]
            entry.append(databyte)
        datalist.append(entry)
        '''
        datalist.append(newdata)

chr_rom_chunk = list()
print(len(datalist))        #Should be 128
for i in datalist:
    chr_rom_chunk.extend(i) #first half of tile
    chr_rom_chunk.extend(i) #second half of tile. It's just a copy.
chr_rom_chunk = chr_rom_chunk + chr_rom_chunk   #Duplicate to form 256 tiles
chr_rom_chunk = chr_rom_chunk + chr_rom_chunk   #Duplicate to form bg+spr
print(len(chr_rom_chunk))   #Should be 8192
prg_rom_chunk = [0 for i in range(16384)]   #dummy data
vectors = [0x00,0xC0,0x00,0xC0,0x00,0xC0]
prg_rom_chunk[-6:] = vectors    #Replaces final six bytes with vector data
prg_rom_chunk[0] = 0x02         #Immediate JAM. ACETONES uses this to exit core.
nes_header = ['N','E','S',0x1A]
VMIRROR = 0
HMIRROR = 1
nes_header.extend([1,1,HMIRROR,0])
nes_header.extend([0,0,0,0,0,0,0,0])

bytedata = tobytes([nes_header, prg_rom_chunk, chr_rom_chunk])
print(f"Should be {16+16384+8192}, is actually: {len(bytedata)}")
print(f"Size check: {'PASS' if 16+16384+8192==len(bytedata) else 'FAIL'}")


filename = "chromdat.nes"
filebase = filename.split('.')[0]

'''with open("nestest.nes","rb") as f:
    data = bytes(f.read())
'''
data = bytedata


print(data[:15])
isrom = False
def lo(num):
    return num & 0xFF
def hi(num):
    return (num >> 8) & 0xFF
if bytes(data[:4]) == bytes([ord('N'),ord('E'),ord('S'), 0x1A]):
    print("Is NES ROM")
    isrom = True
    prg_rom_count = int(data[4])
    chr_rom_count = int(data[5])
    print(f"PRG COUNT {prg_rom_count}, CHR COUNT {chr_rom_count}")
    hashdata = hashlib.sha256(data).digest()[:16]
    print(hashdata)
    datalen = len(data)
    convdata = tobytes(["CENROM", 0, filebase, 0, hashdata, 0, lo(datalen), hi(datalen), data])
    toolkit.export8xv(filebase, convdata)
    if os.path.exists(f"../bin/{filebase}.8xv"):
        os.remove(f"../bin/{filebase}.8xv")
    os.rename(f"{filebase}.8xv",f"../bin/{filebase}.8xv")
    a = (data[6] & 0xF0) >> 4
    b = (data[7] & 0xF0)
    print(f"Mapper ID: {a|b}")



128
8192
Should be 24592, is actually: 24592
Size check: PASS
b'NES\x1a\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00'
Is NES ROM
PRG COUNT 1, CHR COUNT 1
b'ZGBmr\xc1\x9eA\xb1w\xa9\xca\x0e\xaeK\x7f'
Mapper ID: 0


In [4]:
c = 19115
for i in range(1,c):
    if not c%i:
        print(i)
    

1
5
3823


In [11]:
import pyperclip, re

with open("../core.z80", "r") as f:
    lines = [i for i in f]

termcorrelation = {
    "abs": ["absolute"],
    "imm": ["immediate"],
    "zp": ["zeropage"],
    "zpx": ["zeropage,X"],
    "zpy": ["zeropage,Y"],
    "abs": ["absolute"],
    "absx": ["absolute,X","absolut,X"],
    "absy": ["absolute,Y","absolut,Y"],
    "indx": ["(indirect,X)","(indirect,X)"],
    "indy": ["(indirect),Y"],
    #"": "implied",
}

rmwinstructions = [
    "ASL", "DEC", "INC", "LSR", "ROL", "ROR",
    "SLO", "RLA", "SRE", "RRA", "SHA", "DCP", "ISC"
]

twosegmentrwinstructions = [
    "RLA", "RRA", "SLO", "SRE", "DCP", "ISC"
]

class Idata(object):
    def __init__(self, citem:str, pitem:list):
        self.isinit = False
        pitem = [i.strip() for i in pitem]
        try:
            labelcomponents = [i[:-1] if i[-1] == ":" else i for i in citem.split('_')]
            fw = labelcomponents[-1]
            iscorrelated = fw in termcorrelation and pitem[0] in termcorrelation[fw]

            if iscorrelated or pitem[0] in ("implied", "accumulator", "relative", "indirect"):
                self.opcode = int(pitem[2], base=16)
                self.bytes = int(pitem[3])
                self.cycles = re.sub("[^0-9]","",pitem[4])
                prename = pitem[1].split(' ')[0]
                if prename in ("JMP", "JSR"):
                    raise RuntimeError("You've got to handle these ones yourself.")
                if pitem[0] == "accumulator":
                    prename += "A"
                self.name = prename
                self.isinit = True
                self.type = fw.upper() if iscorrelated else ""
                self.linecache = ""
                #print(f"Name: {self.name}, size: {self.bytes}, cycles: {self.cycles}, opcode: {self.opcode:02X}, type: {self.type}")
            else:
                print(f"Check failure on line {citem}, fw={fw}, pitem[0]={pitem}")
                pass
        except:
            if pitem and len(pitem) > 1:
                print(f"Parse failure on {pitem[1]}")
            pass
    @classmethod
    def dirinit(cls, type, name, opcode, bytes, cycles, linecache = ""):
        d = cls("","")
        d.opcode = opcode
        d.bytes = bytes
        d.name = name
        d.type = type
        d.cycles = cycles
        d.linecache = linecache
        d.isinit = True
        return d


found = 0
objs = []
prevline = ""
linecache = ""
for line in lines:
    line = line.strip()
    linecache += line + " "
    cursearch = re.search("^inst6502_.*:$", line)
    prevsearch = re.search("^;.*([0-9]|\*|†)$", prevline)
    if cursearch and prevsearch:
        citem = cursearch.group()
        pitem = prevsearch.group()[1:].split('\t')
        pitem = [i.strip(";") for i in pitem]
        t = Idata(citem, pitem)
        if t.isinit:
            if objs:
                objs[-1].linecache = linecache
                linecache = ""
            objs.append(t)
            found += 1


    #
    prevline = line


illegal_nop_name = "NOPI"    #REPLACE LATER WITH "NOI" WHEN VERIFY IF MEMMODE
#illegal instructions: implied nops
for i in [0x1A, 0x3A, 0x5A, 0x7A, 0xDA, 0xFA]:
    objs.append(Idata.dirinit("", illegal_nop_name, i, 1, 2))
illegal_nop_name = "NOPM"
#illegal instructions: immediate nops
for i in [0x80, 0x82, 0x89, 0xC2, 0xE2]:
    objs.append(Idata.dirinit("IMM", illegal_nop_name, i, 2, 2))
#illegal instructions: zeropage nops
for i in [0x04, 0x44, 0x64]:
    objs.append(Idata.dirinit("ZP", illegal_nop_name, i, 2, 3))
#illegal instructions: zeropage x indexed nops
for i in [0x14, 0x34, 0x54, 0x74, 0xD4, 0xF4]:
    objs.append(Idata.dirinit("ZPX", illegal_nop_name, i, 2, 4))
#illegal instructions: absolute nops
for i in [0x0C,]:
    objs.append(Idata.dirinit("ABS", illegal_nop_name, i, 3, 4,"READ_ACCESS"))
for i in [0x1C, 0x3C, 0x5C, 0x7C, 0xDC, 0xFC]:
    objs.append(Idata.dirinit("ABSX", illegal_nop_name, i, 3, 4,"READ_ACCESS"))

objs.append(Idata.dirinit("IMM", "SBC", 0xEB, 2, 2))
objs.append(Idata.dirinit("", "JSR", 0x20, 3, 6))
objs.append(Idata.dirinit("", "JMPA", 0x4C, 3, 3))
objs.append(Idata.dirinit("", "JMPI", 0x6C, 3, 5))


#illegal instructions: JAMs
for i in [0x02,0x12,0x22,0x32,0x42,0x52,0x62,0x72,0x92,0xB2,0xD2,0xF2]:
    objs.append(Idata.dirinit("", "JAM", i, 1, 4))

listing = dict()
for item in objs:
    listing[item.opcode] = item

print(f"Final listing length: {len(objs)}")

s = ""
memacc = ["NO","R ","W ","RW"]
for index in range(256):
    s += "A6502T("
    if index in listing:
        item = listing[index]
        s += "MEMACC_"
        if not item.type or all([(i not in item.type) for i in ['ABS','IND']]):
            s += "NO, "
            m = -1
        else:
            m = 0
            if "READ_ACCESS" in item.linecache:
                m += 1
            if "WRITE_ACCESS" in item.linecache:
                m += 2
            write_only_names = ["STA", "STY", "STX", "SAX"]
            ineligible_names = rmwinstructions + write_only_names
            if item.name in rmwinstructions and m!=3 and item.type in ["ABS", "ABSX", "ABSY", "INDX", "INDY"]:
                print(f"RMW attribute conflict on instruction {item.opcode:02X}:{item.name} of type {item.type}. Correcting error.")
                m = 3
            elif item.name in write_only_names and m!=2 and item.type in ["ABS", "ABSX", "ABSY", "INDX", "INDY"]:
                print(f"Write-only instruction attribute conflict on instruction {item.opcode:02X}:{item.name} of type {item.type}. Correcting error.")
                m = 2
            elif item.name not in ineligible_names and item.type in ["ABS", "ABSX", "ABSY", "INDX", "INDY"] and m!=1:
                print(f"Read-only instruction attribute conflict on instruction {item.opcode:02X}:{item.name} of type {item.type}. Correcting error.")
                m = 1
            
            s += memacc[m]+", "
        #
        s += "ADRMODE_"
        if not item.type:
            s += "SPEC, "
        else:
            typename = item.type
            ext = ""
            if m > 1 and (typename in ["ABSX","ABSY","INDY"]):
                ext = "F"
            s += (typename+"    ")[:4]+ext+", "
        #
        s += str(item.cycles)+ ", ISC_"+item.name+f")    ;{index:02X}\n"



        pass
    else:
        s += f"MEMACC_, ADRMODE_ , -, ISC_)     ;{index:02X}\n"




print(f"Items found: {found}")

names = list()
specialmoded = list()
sizeof_first_half = 0
for item in objs:
    if item.name in twosegmentrwinstructions:
        if item.name not in specialmoded:
            if not item.type:
                specialmoded.append(item.name)
            else:
                specialmoded.insert(0, item.name)
    else:
        if item.name not in names:
            if not item.type:
                names.append(item.name)
            else:
                names.insert(0, item.name)
                sizeof_first_half += 1

s += f"\nISC_NONSPECIAL_LASTENTRY .equ {sizeof_first_half-1}\n"
s += f"ISC_SPECIAL_NONMODED_LASTENTRY .equ {len(names)-1}\n\n"

for idx, name in enumerate(names):
    s += "ISC_" + name.upper() + f" .equ {idx}\n"
for idx, name in enumerate(specialmoded, len(names)):
    s += "ISC_" + name.upper() + f" .equ {idx}\n"




pyperclip.copy(s)

Parse failure on JMP oper
Parse failure on JMP (oper)
Check failure on line inst6502_jsr:, fw=jsr, pitem[0]=['absolute', 'JSR oper', '20', '3', '6']
Check failure on line inst6502_nop_imm:, fw=imm, pitem[0]=['Bytes:2, Cycles:2, Opcodes: 80, 82, 89, C2, E2']
Check failure on line inst6502_nop_zp:, fw=zp, pitem[0]=['Bytes:2, Cycles:3, Opcodes: 04, 44, 64']
Check failure on line inst6502_nop_zpx:, fw=zpx, pitem[0]=['Bytes:2, Cycles:4, Opcodes: 14, 34, 54, 74, D4, F4']
Final listing length: 256
RMW attribute conflict on instruction 53:SRE of type INDY. Correcting error.
RMW attribute conflict on instruction 93:SHA of type INDY. Correcting error.
Read-only instruction attribute conflict on instruction 9B:TAS of type ABSY. Correcting error.
Read-only instruction attribute conflict on instruction 9C:SHY of type ABSX. Correcting error.
Read-only instruction attribute conflict on instruction 9E:SHX of type ABSY. Correcting error.
RMW attribute conflict on instruction 9F:SHA of type ABSY. Correc